# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import csv
import json
#
# import API key
from api_keys import g_key

import requests
import collections
collections.Iterable = collections.abc.Iterable




### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load the csv, old data
data = pd.read_csv("/Users/siyuanliang/BootCamp_University/Python-api-challenge/WeatherPy/Whether.csv")
data


,Unnamed: 0,City_ID,City,City Country,City date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,1,mount isa,AU,1665334419,-20.7333,139.5000,67.77,21,0,11.50
1,1,2,rikitea,PF,1665334419,-23.1203,-134.9692,71.46,69,10,17.29
2,2,3,ostrovnoy,RU,1665334419,68.0531,39.5131,41.61,79,76,16.28
3,3,4,vierzon,FR,1665334419,47.2101,2.0625,68.38,56,1,4.50
4,4,5,nhamunda,BR,1665334420,-2.1861,-56.7131,104.90,19,84,7.09
...,...,...,...,...,...,...,...,...,...,...,...
573,573,574,hami,CN,1665334566,42.8000,93.4500,54.84,15,43,3.78
574,574,575,parambu,BR,1665334567,-6.2111,-40.6944,97.61,16,6,11.61
575,575,576,hoskote,IN,1665334567,13.0669,77.7983,71.62,100,75,4.61
576,576,577,korla,CN,1665334568,41.7597,86.1469,59.41,26,0,3.51


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)
# Create a list containing coordinates
locations = data[["Lat", "Lng"]]
humidity = data["Humidity"]

# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center=(55.0, -1.0), zoom_level=5)

# Use the Lat and Lng as locations and Humidity as the weight.
# Add the weighted heat map layer to the map
fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity, max_intensity=100))

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#  narrow DataFrame
Newdata = data.loc[(data['Max Temp'] <= 80) & (data['Max Temp'] > 70) & (data['Wind Speed']<10) & (data["Cloudiness"]==0)]
Newdata

,Unnamed: 0,City_ID,City,City Country,City date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
65,65,66,mount darwin,ZW,1665334434,-16.7725,31.5838,74.64,25,0,4.99
92,92,93,fallon,US,1665334441,39.4735,-118.7774,71.92,24,0,4.61
126,126,127,portland,US,1665334334,45.5234,-122.6762,70.77,71,0,4.00
134,134,135,opelousas,US,1665334453,30.5335,-92.0815,77.88,33,0,6.91
141,141,142,bossier city,US,1665334454,32.5160,-93.7321,76.68,43,0,6.91
239,239,240,saint george,US,1665334482,37.1041,-113.5841,75.43,35,0,5.75
243,243,244,beloha,MG,1665334483,-25.1667,45.0500,75.40,62,0,5.86
271,271,272,buin,CL,1665334491,-33.7333,-70.7500,77.86,20,0,3.20
336,336,337,rafaela,AR,1665334484,-31.2503,-61.4867,72.00,6,0,1.99
347,347,348,pimentel,PE,1665334511,-6.8367,-79.9342,70.09,64,0,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Create new plot
# create a heatmap based on the csv


# generate some (latitude, longitude) pairs
fig_hotel_humidity = gmaps.figure()
fig_hotel_humidity.add_layer(gmaps.heatmap_layer(Newdata[["Lat", "Lng"]], weights=Newdata["Humidity"], max_intensity=100))
fig_hotel_humidity

Figure(layout=FigureLayout(height='420px'))

In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{City Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

# get hotel info using Google Place api
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# set parameters to search for hotels
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe

for index, row in Newdata.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response['results']
    # save the hotel name to dataframe
    try:
        print(f"First hotel in {row['City']} is {results[0]['name']}.")

        # store hotel name
        Newdata.loc[index, 'Hotel Name'] = results[0]['name']

        # store hotel coordinates
        Newdata.loc[index, 'Hotel Lat'] = results[0]['geometry']['location']['lat']
        Newdata.loc[index, 'Hotel Lng'] = results[0]['geometry']['location']['lng']


    except (KeyError, IndexError):
        # set empty value
        Newdata.loc[index, 'Hotel Name'] = ""
        Newdata.loc[index, 'Hotel Lat'] = ""
        Newdata.loc[index, 'Hotel Lng'] = ""

        print("Missing field/result... skipping.")
    print("------------")

Newdata

Retrieving Results for Index 65: mount darwin.


/Users/siyuanliang/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/siyuanliang/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing field/result... skipping.
------------
Retrieving Results for Index 92: fallon.
First hotel in fallon is Best Western Fallon Inn & Suites.
------------
Retrieving Results for Index 126: portland.
First hotel in portland is Embassy Suites by Hilton Portland Downtown.
------------
Retrieving Results for Index 134: opelousas.
First hotel in opelousas is Holiday Inn Express & Suites Opelousas, an IHG Hotel.
------------
Retrieving Results for Index 141: bossier city.
First hotel in bossier city is Horseshoe Bossier City.
------------
Retrieving Results for Index 239: saint george.
First hotel in saint george is Best Western Plus Abbey Inn.
------------
Retrieving Results for Index 243: beloha.
Missing field/result... skipping.
------------
Retrieving Results for Index 271: buin.
First hotel in buin is Paihuen.
------------
Retrieving Results for Index 336: rafaela.
First hotel in rafaela is Parra Hotel & Suites.
------------
Retrieving Results for Index 347: pimentel.
First hotel i

,Unnamed: 0,City_ID,City,City Country,City date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
65,65,66,mount darwin,ZW,1665334434,-16.7725,31.5838,74.64,25,0,4.99,,,
92,92,93,fallon,US,1665334441,39.4735,-118.7774,71.92,24,0,4.61,Best Western Fallon Inn & Suites,39.474582,-118.790005
126,126,127,portland,US,1665334334,45.5234,-122.6762,70.77,71,0,4.00,Embassy Suites by Hilton Portland Downtown,45.521883,-122.674165
134,134,135,opelousas,US,1665334453,30.5335,-92.0815,77.88,33,0,6.91,"Holiday Inn Express & Suites Opelousas, an IHG...",30.5164,-92.067826
141,141,142,bossier city,US,1665334454,32.5160,-93.7321,76.68,43,0,6.91,Horseshoe Bossier City,32.515245,-93.738207
239,239,240,saint george,US,1665334482,37.1041,-113.5841,75.43,35,0,5.75,Best Western Plus Abbey Inn,37.089025,-113.584267
243,243,244,beloha,MG,1665334483,-25.1667,45.0500,75.40,62,0,5.86,,,
271,271,272,buin,CL,1665334491,-33.7333,-70.7500,77.86,20,0,3.20,Paihuen,-33.718883,-70.733587
336,336,337,rafaela,AR,1665334484,-31.2503,-61.4867,72.00,6,0,1.99,Parra Hotel & Suites,-31.253823,-61.485728
347,347,348,pimentel,PE,1665334511,-6.8367,-79.9342,70.09,64,0,8.05,Hotel Puerto del Sol,-6.842637,-79.93727


In [11]:
# filter out cities with no hotel# filter out cities with no hotel
Newdata = Newdata.loc[Newdata['Hotel Name'] != ""]

info_content = [info_box_template.format(**row) for index, row in Newdata.iterrows()]

info_content

# Add hotel marker layer on top of heat map using info box template
fig_hotel_humidity.add_layer(gmaps.marker_layer(Newdata[["Hotel Lat", "Hotel Lng"]], info_box_content=info_content))
fig_hotel_humidity


Figure(layout=FigureLayout(height='420px'))